In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
#Todo esto es la inicializacion, siempre necesitaremos uns parkcontext desde el cual empezaremos a trabajar

In [4]:
#Podemos crear un RDD de pares haciendo a mano una lista de tuplas, por ejemplo
lista_tuplas = [(1,'a'),(2,'b'),(3,'c'),(4,'d')]

In [5]:
#hacemos parallelize para crear el RDD
lista_tuplas= sc.parallelize(lista_tuplas)

In [6]:
lista_tuplas.collect()

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]

In [7]:
#el resultado es igual que una lista de tuplas,lo que seria un pair RDD o  RDD Clave_valor


# ZIP
#Otra forma de hacer RDDs de clave valor es usando ZIP, esta funcion comprime en un solo RDD los dos RDD haciendo

#uno clave valor, si no tenemos 2 RDD podemos por ejemplo crear un range para que le ponga numeros como clave


In [18]:
pair_rdd_zip = sc.parallelize(zip(['a','b','c','d','e'],range(5)))
pair_rdd_zip.collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]

In [10]:
#Aclaracion range, si le ponemos solo un numero empieza contando desde 0, si queremos indicarle inicio y final, 
#debemos indicarlo range(3,6) por ejemplo
pair_rdd_zip = sc.parallelize(zip(['a','b','c','d','e'],range(3,7)))
pair_rdd_zip.collect()

[('a', 3), ('b', 4), ('c', 5), ('d', 6)]

In [13]:
#Tambiem prodriamos hacerlo por ejemplo creando una dos rdds 
lista_1 = ['a','b','c','e','f','g','h']
lista_2 = [4,5,6,7,8,9,10]
rdd_con_zip = sc.parallelize(lista_1).zip(sc.parallelize(lista_2)).collect()
rdd_con_zip

[('a', 4), ('b', 5), ('c', 6), ('e', 7), ('f', 8), ('g', 9), ('h', 10)]

In [14]:
#Otro ejemplo, podriamos crearlo usando el tamaño de la lista, y nos daria un valor incremental
rdd_con_zip = sc.parallelize(zip(lista_1,range(len(lista_1)))) #usando el tamaño de la lista

In [15]:
rdd_con_zip.collect()

[('a', 0), ('b', 1), ('c', 2), ('e', 3), ('f', 4), ('g', 5), ('h', 6)]

In [16]:
#Si no queremos que empiece en 0 si no en 1...
rdd_con_zip = sc.parallelize(zip(lista_1,range(1,len(lista_1)+1))) 

In [17]:
rdd_con_zip.collect()

[('a', 1), ('b', 2), ('c', 3), ('e', 4), ('f', 5), ('g', 6), ('h', 7)]

In [54]:
#Tambien podemos crear un RRD clave valor con funciones lambda usando MAP, por ejemplo partiendo de un txt
#Creamos el txt
import os
texto = open("texto_prueba_2.txt", "w")
texto.write("Esto es un texto de prueba."+ '\n' )
texto.write("Para crear RDD de pares"+ '\n' )
texto.write("Desde un fichero de texto"+ '\n' )
texto.write("Estos RDD tienen el formato de Clave valor"+ '\n' )
texto.write("Pueden ser de cualquier tipo de dato"+ '\n' )
texto.write("Por ejemplo (String,int)")
texto.close()
texto = sc.textFile('texto_prueba_2.txt')

In [55]:
texto.takeSample(False,3)

['Desde un fichero de texto',
 'Para crear RDD de pares',
 'Pueden ser de cualquier tipo de dato']

In [60]:
#Usando una funcion lambda creamos el RDD de pares por un lado el numero de caracteres de la linea y por otro la linea en si
texto.map(lambda x: ( len(x),x)).collect()

[(27, 'Esto es un texto de prueba.'),
 (23, 'Para crear RDD de pares'),
 (25, 'Desde un fichero de texto'),
 (42, 'Estos RDD tienen el formato de Clave valor'),
 (36, 'Pueden ser de cualquier tipo de dato'),
 (24, 'Por ejemplo (String,int)')]

# KEYBY() 
#Otra forma de crear un rdd de pares

#Creamoo los valores y le añadimos una clave con la funcion por ejemplo

#Creamos una lista:


In [10]:
lista = ['hola','hello','Adios','Bye']
lista_pair_rdd_keyby = sc.parallelize(lista)

In [69]:
#Ahora a nuestro RDD normal queremos añadirle una key, por ejempl siguiendo con los ejemplos anetriores, el numero de letras,
#usando una funcion lambda y la funcion KeyBy vamos a crear un RDD clave valor, tamaño, palabra
lista_pair_rdd_keyby.keyBy(lambda x: len(x)).collect()

[(4, 'hola'), (5, 'hello'), (5, 'Adios'), (3, 'Bye')]

In [73]:
#En vez del tamaño, por ejemplo por cambiar, vamoss a ponerle como key la primera letra de la palabra
lista_pair_rdd_keyby.keyBy(lambda x: x[0]).collect()

[('h', 'hola'), ('h', 'hello'), ('A', 'Adios'), ('B', 'Bye')]

In [7]:
#Entonces keyby a partir de unos valores creamos las key


# ZIPWITHINDEX
#Esta es otra forma de crear RRD de pares, esto nos da el valor que es igual que el indice, muy facil

#zipWithIndex es una operación que debe realizarse en un solo RDD. Cuando llamamos a este método en un RDD, 

#Spark asignará un índice incremental de tipo Long a cada uno de los elementos y el índice estará en orden ascendente.


In [3]:
rdd_zipindex = sc.parallelize(['b','v','c','x','z','q','w','e','r'])
rdd_zipindex.zipWithIndex().collect()


[('b', 0),
 ('v', 1),
 ('c', 2),
 ('x', 3),
 ('z', 4),
 ('q', 5),
 ('w', 6),
 ('e', 7),
 ('r', 8)]

In [80]:
#Nos crea el indice en funcion del orden del elemento


# zipWithUniqueId()
Tenemos otro similar, que se llama zipWithUniqueId()

Este método también asigna un valor de índice único a cada elemento de un RDD, 

pero no mantendrá ningún orden, ya que asigna un índice mediante Hashing.


In [4]:
rdd_zipindex.zipWithUniqueId().collect()

[('b', 0),
 ('v', 1),
 ('c', 2),
 ('x', 3),
 ('z', 4),
 ('q', 5),
 ('w', 6),
 ('e', 7),
 ('r', 15)]

# ZIP CON dos RDDs
#Esta ya la hemos visto antes
#El método Zip () se utiliza para combinar dos RDD uniendo las filas correspondientes de cada RDD. 

 debemos asegurarnos de que ambos RDD tengan 

#la misma cantidad de elementos, de lo contrario Spark arrojará un error.



In [3]:
rdd1 = sc.parallelize(range(1,8))
rdd2 = sc.parallelize(range(100,107))
pair = rdd1.zip(rdd2)

In [4]:
pair.collect()

[(1, 100), (2, 101), (3, 102), (4, 103), (5, 104), (6, 105), (7, 106)]